In [12]:
import pandas as pd
import datetime

In [13]:
df1 = pd.read_csv("./satellite_breakup(from PDF).csv")
df1[["LAUNCH DATE", "BREAKUP DATE"]] = df1[["LAUNCH DATE", "BREAKUP DATE"]].apply(pd.to_datetime)
df1["launch_designator"] = df1["INTERNATIONAL DESIGNATOR"].str.slice(stop=8)
df1 = df1[['SATELLITE NAME', 'INTERNATIONAL DESIGNATOR', 'US SATELLITE NUMBER', 'LAUNCH DATE', 'BREAKUP DATE', 'DEBRIS CATALOGED', 'DEBRIS ON-ORBIT', 'launch_designator']]

df2 = pd.read_csv("../satcat_all.csv")
df2[["DEBUT", "DECAY", "LAUNCH"]] = df2[["DEBUT", "DECAY", "LAUNCH"]].apply(pd.to_datetime)
df2["launch_designator"] = df2.INTLDES.str.slice(stop=8)
df2 = df2[['INTLDES', 'NORAD_CAT_ID', 'OBJECT_TYPE', 'SATNAME', 'DEBUT', 'LAUNCH', 'DECAY', 'launch_designator']]

In [14]:
def fix_date(x):
    if x.year > 2056:
        year = x.year - 100
    else:
        year = x.year
    return datetime.date(year,x.month,x.day)

# dates < 1969 are parsed as 20xx, fix it
df1['BREAKUP DATE'] = df1['BREAKUP DATE'].apply(fix_date)
df1['LAUNCH DATE'] = df1['LAUNCH DATE'].apply(fix_date)

US SATELLITE NUMBER == NORAD_CAT_ID

Check if PDF data includes anything that doesn't exist in SATCAT

NOPE! That's great

In [43]:
df1[~df1["US SATELLITE NUMBER"].isin(df2.NORAD_CAT_ID.values)]
# 0 rows = everything matches up

,SATELLITE NAME,INTERNATIONAL DESIGNATOR,US SATELLITE NUMBER,LAUNCH DATE,BREAKUP DATE,DEBRIS CATALOGED,DEBRIS ON-ORBIT,launch_designator


As seen below, there are 13 incidents where the same launch had 2 documented breakup dates. Unfortunately we cannot assign the proper breakup date for debris generated from these events. However, there aren't too many of these debris, we may be able to get away with assigning 1 of the breakup dates to all debris with the same starting launch designator.

In [44]:
t = df1.launch_designator.value_counts()
t[t>=2]

2008-067    2
1989-001    2
2002-037    2
1990-045    2
1984-011    2
1990-110    2
1987-079    2
1992-047    2
2010-007    2
1999-057    2
1986-069    2
1988-085    2
1991-025    2
Name: launch_designator, dtype: int64

In [45]:
combined = df2.merge(df1[["launch_designator","BREAKUP DATE"]], how="left", left_on="launch_designator", right_on="launch_designator")
# combined[(combined.OBJECT_TYPE=="DEBRIS") & combined["BREAKUP DATE"].notnull()]
# combined[(combined.OBJECT_TYPE=="DEBRIS") & combined["BREAKUP DATE"].isnull()]

All relevant data is now combined int he `combined` dataframe.  We just need to select the correct date to use with the following rules:
1. For non-DEBRIS, we can use the LAUNCH
1. For debris listed in the PDF file, we should use `BREAKUP DATE`
1. If the `DEBUT` is earlier than `DECAY`, we can probably trust it
1. If it `DEBUT` later than 2004, we can also trust it since these are reported to space track as soon as they are spotted
1. if a debris `DECAYED` within 1 year of `LAUNCH`, let's just assumed that it was created on launch, ok? :)

In [46]:
# For non-DEBRIS, we can use the LAUNCH
combined.loc[(combined.OBJECT_TYPE!="DEBRIS"),"exist_date"] = combined.LAUNCH

In [47]:
# For debris listed in the PDF file, we should use BREAKUP DATE
combined.loc[((combined.OBJECT_TYPE=="DEBRIS") & combined["BREAKUP DATE"].notnull()),"exist_date"] = combined["BREAKUP DATE"]

In [48]:
# If the `DEBUT` is earlier than `DECAY`, we can probably trust it
combined.loc[((combined.OBJECT_TYPE=="DEBRIS") & combined["exist_date"].isnull() & (combined.DECAY > combined.DEBUT)),"exist_date"] = combined.DEBUT

In [50]:
# If it `DEBUT` later than 2004, we can also trust it since these are reported to space track as soon as they are spotted
combined.loc[((combined.OBJECT_TYPE=="DEBRIS") & combined["exist_date"].isnull() & (combined.DEBUT > "2004")),"exist_date"] = combined.DEBUT

In [59]:
# if a debris DECAYED within 1 year, let's just assumed that it was created on launch, ok? :)
combined.loc[(combined["exist_date"].isnull() & (combined.DECAY - combined.LAUNCH < datetime.timedelta(days=365))),"exist_date"] = combined.LAUNCH

In [60]:
combined[combined["exist_date"].isnull()]

,INTLDES,NORAD_CAT_ID,OBJECT_TYPE,SATNAME,DEBUT,LAUNCH,DECAY,launch_designator,BREAKUP DATE,exist_date
16,1959-001C,5807,DEBRIS,VANGUARD DEB,2002-01-13,1959-02-17,1982-11-22,1959-001,NaN,NaT
49,1960-005D,37,DEBRIS,SPUTNIK 4 DEB,2002-01-13,1960-05-15,1961-06-30,1960-005,NaN,NaT
76,1960-014C,69,DEBRIS,JUNO II DEB,2002-01-13,1960-11-03,1970-02-16,1960-014,NaN,NaT
77,1960-014D,105,DEBRIS,JUNO II DEB,2002-01-13,1960-11-03,1972-03-03,1960-014,NaN,NaT
101,1961-004E,3738,DEBRIS,SCOUT X-1 DEB,2002-01-13,1961-02-16,1982-03-29,1961-004,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
35964,1999-028G,25751,DEBRIS,USA 144 DEB,2002-01-13,1999-05-22,NaT,1999-028,NaN,NaT
35966,1999-028J,25753,DEBRIS,USA 144 DEB,2002-01-13,1999-05-22,NaT,1999-028,NaN,NaT
35967,1999-028K,25754,DEBRIS,USA 144 DEB,2002-01-13,1999-05-22,NaT,1999-028,NaN,NaT
35968,1999-028L,25755,DEBRIS,USA 144 DEB,2002-01-13,1999-05-22,NaT,1999-028,NaN,NaT


In [8]:
display(df1[df1["INTERNATIONAL DESIGNATOR"]=="1961-015C"])
print("Documented in satcat:",len(df2[df2.INTLDES.str.startswith("1961-015C") & (df2.OBJECT_TYPE == "DEBRIS")]))
# can see from this example that the PDF file DEBRIS CATALOGED and DEBRIS ON-ORBIT is more than SATCAT

,SATELLITE NAME,INTERNATIONAL DESIGNATOR,US SATELLITE NUMBER,LAUNCH DATE,BREAKUP DATE,DEBRIS CATALOGED,DEBRIS ON-ORBIT,launch_designator
0,TRANSIT 4A R/B,1961-015C,118,1961-06-29,1961-06-29,296,172.0,1961-015


Documented in satcat: 24


In [6]:
df2.loc[(df2.OBJECT_TYPE!="DEBRIS"),"exist_date"] = df2[["DEBUT", "LAUNCH"]].min(axis=1)